<a href="https://colab.research.google.com/github/jumbokh/csu1112-class/blob/main/notebooks/flags/F2379_CH11_PART04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#第11章：文字資料的深度學習

##11-5 文字分類以外的任務 - 以Seq2seq模型為例

###11-5-1 機器翻譯的案例

In [ ]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2022-05-25 05:40:08--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.188.208, 142.251.33.208, 172.217.15.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.188.208|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.008s  

2022-05-25 05:40:08 (297 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [ ]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [ ]:
import random
print(random.choice(text_pairs))

("I'm busy today, otherwise I would accept.", '[start] Estoy ocupado hoy, de otra manera, aceptaría. [end]')


In [ ]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

####程式 11.26 向量化英文和西文的文字組合

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import string
import re

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

####程式 11.27 為翻譯任務準備資料集

In [ ]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


###11-5-2 用RNN進行Seq2seq的學習

####程式 11.28 使用GRU編碼器

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

####程式 11.29 使用GRU解碼器及端到端模型

In [ ]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

####程式 11.30 訓練循環Seq2seq模型

In [ ]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
1302/1302 [==============================] - 138s 94ms/step - loss: 1.6336 - accuracy: 0.4180 - val_loss: 1.3128 - val_accuracy: 0.5072
Epoch 2/15
1302/1302 [==============================] - 119s 91ms/step - loss: 1.3169 - accuracy: 0.5244 - val_loss: 1.1544 - val_accuracy: 0.5678
Epoch 3/15
1302/1302 [==============================] - 119s 91ms/step - loss: 1.1724 - accuracy: 0.5750 - val_loss: 1.0736 - val_accuracy: 0.5994
Epoch 4/15
1302/1302 [==============================] - 119s 91ms/step - loss: 1.0803 - accuracy: 0.6088 - val_loss: 1.0299 - val_accuracy: 0.6205
Epoch 5/15
1302/1302 [==============================] - 118s 91ms/step - loss: 1.0311 - accuracy: 0.6342 - val_loss: 1.0193 - val_accuracy: 0.6303
Epoch 6/15
1302/1302 [==============================] - 119s 91ms/step - loss: 1.0024 - accuracy: 0.6521 - val_loss: 1.0156 - val_accuracy: 0.6359
Epoch 7/15
1302/1302 [==============================] - 119s 91ms/step - loss: 0.9822 - accuracy: 0.6664 - val_loss: 1

####程式 11.31 用RNN編碼器和解碼器翻譯新的句子

In [ ]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Tom dove into the pool.
[start] tom se [UNK] en la la que la estaba una [UNK] [end]
-
I don't think that Tom realizes how rude he's being.
[start] no creo que tom sea lo que [UNK] de tom [end]
-
Where did they learn this?
[start] dónde lo hizo esto [end]
-
It is important for me.
[start] es importante para mí [end]
-
Tom suffered from migraine headaches for years.
[start] tom se [UNK] de años de años [end]
-
Where did the accident take place?
[start] en qué lugar ese accidente [end]
-
Are you pleased with your new house?
[start] estás [UNK] con tu nueva casa [end]
-
I never forget a name.
[start] nunca se te he nombre un nombre [end]
-
Anything you say can and will be used against you.
[start] algo que puede decir y [UNK] con vos [end]
-
I'm touched you remembered.
[start] te estoy [UNK] [end]
-
He's completely crazy.
[start] está completamente loco [end]
-
Tom didn't have anything to worry about.
[start] tom no tenía nada de qué hacer [end]
-
Get out of the classroom.
[start] [UNK] 

###11-5-3 用Transfomer進行Seq2seq學習

####程式 11.33 TransformerDecoder
####程式 11.34 生成因果遮罩(casual mask)的方法
####程式 11.35 TransformerDecoder的正向傳遞

In [ ]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

####程式 11.36 端到端Transformer

In [ ]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

####程式 11.37 訓練Seq2seq Transformer

In [ ]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1302/1302 [==============================] - 98s 73ms/step - loss: 1.6668 - accuracy: 0.4210 - val_loss: 1.3262 - val_accuracy: 0.5132
Epoch 2/30
1302/1302 [==============================] - 94s 72ms/step - loss: 1.3348 - accuracy: 0.5350 - val_loss: 1.1679 - val_accuracy: 0.5691
Epoch 3/30
1302/1302 [==============================] - 94s 72ms/step - loss: 1.1836 - accuracy: 0.5833 - val_loss: 1.0812 - val_accuracy: 0.6022
Epoch 4/30
1302/1302 [==============================] - 95s 73ms/step - loss: 1.0937 - accuracy: 0.6161 - val_loss: 1.0335 - val_accuracy: 0.6268
Epoch 5/30
1302/1302 [==============================] - 94s 72ms/step - loss: 1.0456 - accuracy: 0.6375 - val_loss: 1.0099 - val_accuracy: 0.6376
Epoch 6/30
1302/1302 [==============================] - 94s 72ms/step - loss: 1.0141 - accuracy: 0.6531 - val_loss: 0.9953 - val_accuracy: 0.6463
Epoch 7/30
1302/1302 [==============================] - 94s 72ms/step - loss: 0.9918 - accuracy: 0.6658 - val_loss: 0.9899 -

####程式 11.38 用Transformer模型翻譯新句子

In [ ]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
They both want to say something.
[start] ellos dos quiero decir algo [end]
-
Many scientists knew him.
[start] muchos invitó a él [end]
-
Tom knows what you did.
[start] tom sabe lo que tú has hecho [end]
-
He is now almost as tall as his father is.
[start] ahora él es casi tan alto como su padre [end]
-
It doesn't hurt as much as it did before.
[start] no pasa tanto como antes [end]
-
I'm ashamed to go out dressed like this.
[start] estoy avergonzado de que te [UNK] esto [end]
-
He is better off than he was ten years ago.
[start] Él es mejor de lo que fue hace diez años [end]
-
I didn't walk.
[start] yo no [UNK] [end]
-
Tom rinsed off the soap.
[start] tom se [UNK] el [UNK] [end]
-
I paid her five dollars.
[start] le yo sabía cinco dólares [end]
-
Let's shake hands and be friends.
[start] vamos [UNK] a tener amigos [end]
-
Don't make fun of them.
[start] no te hagas [UNK] [end]
-
It's there.
[start] está allí [end]
-
My wife is a good cook.
[start] mi esposa es una buena vista [end]